We'll use this notebook to prep our New York Airbnb data.

In [3]:
#import packages
import pandas as pd

pd.set_option('display.max_columns',None)

import requests
from tqdm import tqdm
tqdm.pandas()

from time import sleep
# import geopy

First, we'll import the raw data and take a look

In [4]:
#import and view data
ny_df = pd.read_csv('AB_NYC_2019.csv')

ny_df

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48890,36484665,Charming one bedroom - newly renovated rowhouse,8232441,Sabrina,Brooklyn,Bedford-Stuyvesant,40.67853,-73.94995,Private room,70,2,0,NaN,NaN,2,9
48891,36485057,Affordable room in Bushwick/East Williamsburg,6570630,Marisol,Brooklyn,Bushwick,40.70184,-73.93317,Private room,40,4,0,NaN,NaN,2,36
48892,36485431,Sunny Studio at Historical Neighborhood,23492952,Ilgar & Aysel,Manhattan,Harlem,40.81475,-73.94867,Entire home/apt,115,10,0,NaN,NaN,1,27
48893,36485609,43rd St. Time Square-cozy single bed,30985759,Taz,Manhattan,Hell's Kitchen,40.75751,-73.99112,Shared room,55,1,0,NaN,NaN,6,2


This data looks pretty good at first glance, but we don't need all of these columns. Let's only select the data we actually want.

In [5]:
#select desired data
ny_drop = ny_df[
    [
        "id",
        "host_id",
        "neighbourhood_group",
        "neighbourhood",
        "latitude",
        "longitude",
        "room_type",
        "price",
        "minimum_nights",
        "number_of_reviews",
        "reviews_per_month",
        "availability_365",
    ]
]

ny_drop

,id,host_id,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,availability_365
0,2539,2787,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,0.21,365
1,2595,2845,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,0.38,355
2,3647,4632,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,365
3,3831,4869,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,4.64,194
4,5022,7192,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,0.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...
48890,36484665,8232441,Brooklyn,Bedford-Stuyvesant,40.67853,-73.94995,Private room,70,2,0,NaN,9
48891,36485057,6570630,Brooklyn,Bushwick,40.70184,-73.93317,Private room,40,4,0,NaN,36
48892,36485431,23492952,Manhattan,Harlem,40.81475,-73.94867,Entire home/apt,115,10,0,NaN,27
48893,36485609,30985759,Manhattan,Hell's Kitchen,40.75751,-73.99112,Shared room,55,1,0,NaN,2


It looks like there's a lot of null values in the reviews_per_month column. Let's see how much missing data we're dealing with.

In [6]:
len(ny_drop[ny_drop['reviews_per_month'].isnull()]) / len(ny_drop)

0.20558339298496778

Yikes, it looks like 20% of reviews_per_month is missing. I wanted to keep this column just in case it became useful later on in exploration, but with 20% missing data, we might as well just drop it.

In [7]:
ny_drop1 = ny_drop.drop(columns='reviews_per_month')
ny_drop1

,id,host_id,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,availability_365
0,2539,2787,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,365
1,2595,2845,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,355
2,3647,4632,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,365
3,3831,4869,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,194
4,5022,7192,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,0
...,...,...,...,...,...,...,...,...,...,...,...
48890,36484665,8232441,Brooklyn,Bedford-Stuyvesant,40.67853,-73.94995,Private room,70,2,0,9
48891,36485057,6570630,Brooklyn,Bushwick,40.70184,-73.93317,Private room,40,4,0,36
48892,36485431,23492952,Manhattan,Harlem,40.81475,-73.94867,Entire home/apt,115,10,0,27
48893,36485609,30985759,Manhattan,Hell's Kitchen,40.75751,-73.99112,Shared room,55,1,0,2


This looks great, however, in order to join with the us_realtor data, we'll need a zipcode for each listing. 

Luckily, we already have the latitude and longitude coordinates. We'll need to connect to an online database to convert them into zipcodes.

This will take quite a while, so I don't recommend repeating this step.

In [8]:
def get_zipcode(df, lat, lon):
    url = f'https://nominatim.openstreetmap.org/reverse?lat={df[lat]}&lon={df[lon]}&format=json&namedetails=1'#&accept-language=en'
    result = requests.get(url=url)
    result_json = result.json()
    sleep(.5)
    try:
        return result_json['address']['postcode']
    except:
        return None

As soon as the zipcodes have been collected, we'll store them in a csv to make sure we don't have to repeat the last step at any point.

In [9]:
# write zipcodes to csv for safekeeping

def write_zips(n, zip_df):
    filepath = f"/nyzips/us_realtor_zips{n}.csv"

    zip_df.to_csv(
        filepath,
        header="column_names",
    )

I kept running into errors, so I decided batch my requests into groups of 2000, saving each group to a different csv so I didn't lose my progress at every error.

In [10]:
for i in range(0,25):
    zipcodes = ny_drop1[i*2000:((i+1)*2000)-1].progress_apply(get_zipcode, axis=1, lat='latitude', lon='longitude')
    write_zips(i,zipcodes)

Now that we've successfully recorded our zipcodes into numerous csvs, we'll need to read them in and append them to a single zipcode dataframe.

In [27]:
#initialize zip_df with first csv
zip_df = pd.DataFrame()

#iterate through the 24 zipcode csvs and append them to the df
for i in range(0,25):
    zip = pd.read_csv(f"/nyzips/us_realtor_zips{i}.csv")
    # print(zip)
    zip_df = pd.concat([zip_df,zip], axis=0)

# set the index so we don't have extra columns
zip_df.set_index('Unnamed: 0',drop=True, inplace=True)

zip_df

,0
Unnamed: 0,
0,11218.0
1,10018.0
2,10027.0
3,11238.0
4,10029.0
...,...
48890,11216.0
48891,11206.0
48892,10027.0


Notice how we actually have less rows than our original dataset. We want to solve this before appending our zipcode data, or else we risk mismatching zipcodes and ruining our data!

Also notice how our new index seems to be correct for each entry. It looks like we missed a few zipcodes, with our request returning None. 

That's okay. We'll just fill in the missing indices to get a full-length dataframe.

In [29]:
#fill in missing index values
zip_fill = zip_df.reindex(list(range(zip_df.index.min(),zip_df.index.max()+1)))

zip_fill

,0
Unnamed: 0,
0,11218.0
1,10018.0
2,10027.0
3,11238.0
4,10029.0
...,...
48890,11216.0
48891,11206.0
48892,10027.0


Great, now that the length is correct, we can append our zipcodes to our original dataframe. We'll also cast them as ints to get rid of the floating zeros.

In [35]:
# add zipcodes to their corresponding listings
ny_drop1['zipcode'] = zip_fill.astype('Int64')

ny_drop1

,id,host_id,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,availability_365,zipcode
0,2539,2787,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,365,11218
1,2595,2845,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,355,10018
2,3647,4632,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,365,10027
3,3831,4869,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,194,11238
4,5022,7192,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,0,10029
...,...,...,...,...,...,...,...,...,...,...,...,...
48890,36484665,8232441,Brooklyn,Bedford-Stuyvesant,40.67853,-73.94995,Private room,70,2,0,9,11216
48891,36485057,6570630,Brooklyn,Bushwick,40.70184,-73.93317,Private room,40,4,0,36,11206
48892,36485431,23492952,Manhattan,Harlem,40.81475,-73.94867,Entire home/apt,115,10,0,27,10027
48893,36485609,30985759,Manhattan,Hell's Kitchen,40.75751,-73.99112,Shared room,55,1,0,2,10036


Now that everything's together, we'll just drop the rows with missing zipcodes, since we'll be unable to connect them to the rest of our data anyway.

In [37]:
#drop the missing zipcodes, as the data is useless without the zip
ny_airbnb = ny_drop1.dropna(subset='zipcode')

ny_airbnb

,id,host_id,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,availability_365,zipcode
0,2539,2787,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,365,11218
1,2595,2845,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,355,10018
2,3647,4632,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,365,10027
3,3831,4869,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,194,11238
4,5022,7192,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,0,10029
...,...,...,...,...,...,...,...,...,...,...,...,...
48890,36484665,8232441,Brooklyn,Bedford-Stuyvesant,40.67853,-73.94995,Private room,70,2,0,9,11216
48891,36485057,6570630,Brooklyn,Bushwick,40.70184,-73.93317,Private room,40,4,0,36,11206
48892,36485431,23492952,Manhattan,Harlem,40.81475,-73.94867,Entire home/apt,115,10,0,27,10027
48893,36485609,30985759,Manhattan,Hell's Kitchen,40.75751,-73.99112,Shared room,55,1,0,2,10036


This listing data looks great now. Let's save it to a new csv to be used in further exploration.

In [38]:
#save data to csv
filepath = "ny_airbnb_prep.csv"

ny_airbnb.to_csv(
    filepath,
    header="column_names",
)